# 1.12 Integration

FriCAS has extensive library facilities for integration.

The first example is the integration of a fraction with denominator that factors into a quadratic and a quartic irreducible polynomial. The usual partial fraction approach used by most other computer algebra systems either fails or introduces expensive unneeded algebraic numbers.

We use a factorization-free algorithm. 

In [1]:
integrate((x^2+2*x+1)/((x+1)^6+1),x)

         3     2
   atan(x  + 3x  + 3x + 1)
   -----------------------
              3
                                         Type: Union(Expression(Integer),...)


When real parameters are present, the form of the integral can depend on the signs of some expressions.

Rather than query the user or make sign assumptions, FriCAS returns all possible answers. 

In [2]:
integrate(1/(x^2 + a),x)

          2      +---+
        (x  - a)\|- a  + 2a x         +-+
    log(---------------------)      x\|a
                 2             atan(-----)
                x  + a                a
   [--------------------------,-----------]
                +---+               +-+
              2\|- a               \|a
                                   Type: Union(List(Expression(Integer)),...)


The `integrate` operation generally assumes that __all__ parameters are real. The only exception is when the integrand has complex valued quantities.

If the parameter is complex instead of real, then the notion of _sign_ is undefined and there is a _unique_ answer. You can request this answer by prepending the word _complex_ to the command name:

In [3]:
complexIntegrate(1/(x^2 + a),x)

    +---+      +---+         +---+        +---+
    |  1       |  1          |  1         |  1
    |- - log(a |- -  + x) -  |- - log(- a |- -  + x)
   \|  a      \|  a         \|  a        \|  a
   -------------------------------------------------
                           2
                                                    Type: Expression(Integer)


The following two examples illustrate the limitations of table-based approaches. The two integrands are very similar, but the answer to one of them requires the addition of two new algebraic numbers.

This one is the easy one. The next one looks very similar but the answer is much more complicated. 

In [4]:
integrate(x^3 / (a+b*x)^(1/3),x)

        3 3         2 2       2          3 3+-------+2
   (120b x  - 135a b x  + 162a b x - 243a )\|b x + a
   ---------------------------------------------------
                              4
                          440b
                                         Type: Union(Expression(Integer),...)


Only an algorithmic approach is guaranteed to find what new constants must be added in order to find a solution. 

In [6]:
integrate(1 / (x^3 * (a+b*x)^(1/3)),x)

           2 2 +-+    3+-+3+-------+2   3+-+2 3+-------+
       - 2b x \|3 log(\|a \|b x + a   + \|a   \|b x + a + a)
     + 
         2 2 +-+    3+-+2 3+-------+
       4b x \|3 log(\|a   \|b x + a - a)
     + 
                  +-+3+-+2 3+-------+    +-+
        2 2     2\|3 \|a   \|b x + a + a\|3                  +-+3+-+3+-------+2
     12b x atan(----------------------------) + (12b x - 9a)\|3 \|a \|b x + a
                             3a
  /
        2 2 +-+3+-+
     18a x \|3 \|a
                                         Type: Union(Expression(Integer),...)


Some computer algebra systems use heuristics or table-driven approaches to integration. When these systems cannot determine the answer to an integration problem, they reply `I don't know`. FriCAS uses an algorithm which is a __decision procedure__ for integration. If FriCAS returns the original integral that __conclusively proves__ that an integral cannot be expressed in terms of elementary functions.

When FriCAS returns an integral sign, it has proved that no answer exists as an elementary function. 

In [7]:
integrate(log(1 + sqrt(a*x + b)) / x,x)

      x      +--------+
    ++  log(\|b + %G a  + 1)
    |   -------------------- d%G
   ++            %G
                                         Type: Union(Expression(Integer),...)


FriCAS can handle complicated mixed functions much beyond what you can find in tables.

Whenever possible, FriCAS tries to express the answer using the functions present in the integrand. 

In [8]:
integrate((sinh(1+sqrt(x+b))+2*sqrt(x+b)) / (sqrt(x+b) * (x + cosh(1+sqrt(x + b)))), x)

                        +-----+
               - 2cosh(\|x + b  + 1) - 2x            +-----+
   2log(---------------------------------------) - 2\|x + b
              +-----+              +-----+
        sinh(\|x + b  + 1) - cosh(\|x + b  + 1)
                                         Type: Union(Expression(Integer),...)


A strong _structure-checking_ algorithm in FriCAS finds hidden algebraic relationships between functions. 

In [9]:
integrate(tan(atan(x)/3),x)

             atan(x) 2             atan(x) 2           atan(x)
   8log(3tan(-------)  - 1) - 3tan(-------)  + 18x tan(-------)
                3                     3                   3
   ------------------------------------------------------------
                                18
                                         Type: Union(Expression(Integer),...)


The _discovery_ of this algebraic relationship is necessary for correct integration of this function. Here are the details: 

1.  If $x=tan(t)$ and $g=tan(\frac{t}{3})$ then the following algebraic relation is true:
    $$ g^3-3x g^2-3g+x=0 $$
2.  Integrate $g$ using this algebraic relation; this produces: 
    $$\frac{(24 g^2-8)\,log(3 g^2 - 1)+(81x^2+24)g^2+72 x g-27 x^2-16}{54 g^2-18}$$
3   Rationalize the denominator, producing:
    $$\frac{8\log(3g^2-1)-3 g^2+18 x g+16}{18}$$
    Replace $g$ by the initial definition $g=\tan(\arctan(x)/3)$ to produce the final result. 
    
This is an example of a mixed function where the algebraic layer is over the transcendental one. 

In [10]:
integrate((x + 1) / (x*(x + log x) ^ (3/2)), x)

       +----------+
     2\|log(x) + x
   - --------------
       log(x) + x
                                         Type: Union(Expression(Integer),...)


While incomplete for non-elementary functions, FriCAS can handle some of them. 

In [11]:
integrate(exp(-x^2) * erf(x) / (erf(x)^3 - erf(x)^2 - erf(x) + 1),x)

                +---+    erf(x) - 1      +---+
   (erf(x) - 1)\|%pi log(----------) - 2\|%pi
                         erf(x) + 1
   -------------------------------------------
                   8erf(x) - 8
                                         Type: Union(Expression(Integer),...)


More examples of FriCAS's integration capabilities are discussed in section [ugProblemIntegration](). 